# Sentence Pair Classification

In [1]:
import os
import sys
import logging
import torch
import numpy as np
import argparse
from math import exp

from torch import nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim

from random import random as rand


from transformers import BertTokenizer, BertForNextSentencePrediction
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModel
#import sentence_pair_utils
#import sentence_pair_evaluation
import transformers
import pandas as pd

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

import random

cuda


In [2]:
transformers.logging.set_verbosity_error()

LOG_FORMAT = '%(asctime)s : %(filename)s : %(funcName)s : %(levelname)s : %(message)s'
logging.basicConfig(filename='SentencePairCLS.log', level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("SentencePairCLS")

In [3]:
import pickle
def pkl_vars(varname, filename):
    with open(filename, 'wb') as file:
        pickle.dump(varname, file)
        
def reload_vars(filename):
    this_var = None
    with open(filename, 'rb') as file:
        this_var = pickle.load(file)      
    return this_var

## Dataset prepation

In [125]:
df_flattened_dd = pd.read_csv("parsed_disambiguation_list_without===.csv")
df_flattened_dd.tail()

,dd_page_title,term,definition,def_page_title,categories,plain_text_definition
1976,Definition:Zero,Zero,Let $\mathbb A$ be one of the standard number ...,Definition:Zero Mapping,['Definitions/Mapping Theory'],"Let 𝔸 be one of the standard number systems ℕ,..."
1977,Definition:Zero,Zero,Let $f: R \to R$ be a mapping on a ring $R$.\n...,Definition:Root of Mapping,"['Definitions/Roots of Mappings', 'Definitions...",Let f: R → R be a mapping on a ring R.\n\nLet ...
1978,Definition:Zero,Zero,"Let $\left( R, +_R, \times_R \right)$ be a rin...",Definition:Zero Vector,"['Definitions/Module Theory', 'Definitions/Vec...","Let ( R, +_R, ×_R ) be a ring.\n\nLet ( G, +_G..."
1979,Definition:Zero Locus,Zero Locus,Let $k$ be a field.\n\nLet $n\geq1$ be a natur...,Definition:Zero Locus of Set of Polynomials,['Definitions/Algebraic Geometry'],Let k be a field.\n\nLet n≥1 be a natural numb...
1980,Definition:Zero Locus,Zero Locus,Let $A$ be a commutative ring with unity.\n\nL...,Definition:Vanishing Set of Subset of Ring,['Definitions/Zariski Topology'],Let A be a commutative ring with unity.\n\nLet...


In [139]:
df_flattened_dd['term_in_def'] = df_flattened_dd.apply(lambda row: row['term'].lower() in row['plain_text_definition'].lower(), axis=1)
df_flattened_dd[~df_flattened_dd['term_in_def']]

,dd_page_title,term,definition,def_page_title,categories,plain_text_definition,term_in_def,biased_indexes,def_contains_other_def
4,Definition:Absolute,Absolute,A constant is a name for an object (usually a ...,Definition:Constant,"['Definitions/Constants', 'Definitions/Algebra']",A constant is a name for an object (usually a ...,False,False,False
12,Definition:Absolute,Absolute,Let $S$ be a sample or a population.\n\nLet $\...,Definition:Frequency (Descriptive Statistics),['Definitions/Frequency (Descriptive Statistic...,Let S be a sample or a population.\n\nLet ω be...,False,False,True
29,Definition:Additive Function,Additive Function,Let $R$ be a unique factorization domain.\n\nL...,Definition:Additive Function on UFD,['Definitions/Ring Theory'],Let R be a unique factorization domain.\n\nLet...,False,False,False
32,Definition:Additive Function,Additive Function,Let $\unicode{x3a3}$ be a $\sigma$-algebra.\n\...,Definition:Countably Additive Function,"['Definitions/Set Systems', 'Definitions/Measu...",Let x3a3 be a σ-algebra.\n\nLet f: x3a3→R be a...,False,False,False
51,Definition:Adjacent Faces,Adjacent Faces,":\n\nLet $G = \left( V, E \right)$ be a planar...",Definition:Adjacent (Graph Theory)/Faces,"['Definitions/Adjacency (Graph Theory)', 'Defi...",":\n\nLet G = ( V, E ) be a planar graph.\n\nTw...",False,False,True
...,...,...,...,...,...,...,...,...,...
1941,Definition:Vertex,Vertex,Consider a cone consisting of the set of all s...,Definition:Cone (Geometry)/Apex,['Definitions/Cones'],Consider a cone consisting of the set of all s...,False,False,False
1953,Definition:Weak,Weak,Let $X$ be a set.\n\nLet $I$ be an indexing se...,Definition:Initial Topology,"['Definitions/Examples of Topologies', 'Defini...",Let X be a set.\n\nLet I be an indexing set.\n...,False,False,False
1962,Definition:Word,Word,Let $n \in \mathbb N$ be a natural number.\n\n...,Definition:Ordered Tuple,"['Definitions/Set Theory', 'Definitions/Mappin...",Let n ∈ℕ be a natural number.\n\nLet ℕ^*_n be ...,False,False,False
1977,Definition:Zero,Zero,Let $f: R \to R$ be a mapping on a ring $R$.\n...,Definition:Root of Mapping,"['Definitions/Roots of Mappings', 'Definitions...",Let f: R → R be a mapping on a ring R.\n\nLet ...,False,False,True


In [140]:
# Todo: does it really matter ? 
df_flattened_dd['term_in_def_page_title'] = df_flattened_dd.apply(lambda row: row['term'].lower() in row['def_page_title'].lower(), axis=1)
df_flattened_dd[~df_flattened_dd['term_in_def_page_title']]

,dd_page_title,term,definition,def_page_title,categories,plain_text_definition,term_in_def,biased_indexes,def_contains_other_def,term_in_def_page_title
4,Definition:Absolute,Absolute,A constant is a name for an object (usually a ...,Definition:Constant,"['Definitions/Constants', 'Definitions/Algebra']",A constant is a name for an object (usually a ...,False,False,False,False
12,Definition:Absolute,Absolute,Let $S$ be a sample or a population.\n\nLet $\...,Definition:Frequency (Descriptive Statistics),['Definitions/Frequency (Descriptive Statistic...,Let S be a sample or a population.\n\nLet ω be...,False,False,True,False
51,Definition:Adjacent Faces,Adjacent Faces,":\n\nLet $G = \left( V, E \right)$ be a planar...",Definition:Adjacent (Graph Theory)/Faces,"['Definitions/Adjacency (Graph Theory)', 'Defi...",":\n\nLet G = ( V, E ) be a planar graph.\n\nTw...",False,False,True,False
52,Definition:Adjacent Faces,Adjacent Faces,Two faces of a polyhedron that meet at the sam...,Definition:Polyhedron/Adjacent/Faces,['Definitions/Adjacent (Polyhedra)'],Two faces of a polyhedron that meet at the sam...,False,False,True,False
70,Definition:Aggregation,Aggregation,Parenthesis is a syntactical technique to disa...,Definition:Parenthesis,"['Definitions/Parenthesis', 'Definitions/Symbo...",Parenthesis is a syntactical technique to disa...,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1941,Definition:Vertex,Vertex,Consider a cone consisting of the set of all s...,Definition:Cone (Geometry)/Apex,['Definitions/Cones'],Consider a cone consisting of the set of all s...,False,False,False,False
1953,Definition:Weak,Weak,Let $X$ be a set.\n\nLet $I$ be an indexing se...,Definition:Initial Topology,"['Definitions/Examples of Topologies', 'Defini...",Let X be a set.\n\nLet I be an indexing set.\n...,False,False,False,False
1962,Definition:Word,Word,Let $n \in \mathbb N$ be a natural number.\n\n...,Definition:Ordered Tuple,"['Definitions/Set Theory', 'Definitions/Mappin...",Let n ∈ℕ be a natural number.\n\nLet ℕ^*_n be ...,False,False,False,False
1977,Definition:Zero,Zero,Let $f: R \to R$ be a mapping on a ring $R$.\n...,Definition:Root of Mapping,"['Definitions/Roots of Mappings', 'Definitions...",Let f: R → R be a mapping on a ring R.\n\nLet ...,False,False,True,False


In [147]:
todo_dict_term_not_in_def = df_flattened_dd[~df_flattened_dd['term_in_def']].to_dict(orient='records')
pkl_vars(todo_dict_term_not_in_def, 'data/vars/todo_dict_term_not_in_def')

### todo1: use the redirections to repair the definition titles containing term
e.g.: https://proofwiki.org/wiki/Definition:Frequency_(Descriptive_Statistics) does not contain "Abosolute"  https://proofwiki.org/wiki/Definition:Absolute_Frequency https://proofwiki.org/wiki/Definition:Absolute

In [ ]:
redirects = reload_vars('data/vars/redirects_list.pkl')

In [ ]:
redirects.get('Definition:Absolute Frequency')

'Definition:Frequency (Descriptive Statistics)'

In [167]:
inv_redirects = {}
for k, v in redirects.items():
    redirects[v] = redirects.get(v, []) + [k]

RuntimeError: dictionary changed size during iteration

### todo2: use the redirection title pairs make positive pairs

### Bias removal: An upper category term of which the definition contains the definitions of its sub-category terms should not be in the same group with these sub-category terms 

In [127]:
contained_indexes = []
duplicated_upper_indexes = []
def_list = list(df_flattened_dd["plain_text_definition"])
term_list = list(df_flattened_dd["term"])
for index1, def1 in enumerate(def_list):
    for index2, def2 in enumerate(def_list):
        if (index2 != index1 and def1 in def2):
            contained_indexes.append((index1, index2))
            if term_list[index1] == term_list[index2]:
                duplicated_upper_indexes.append((index1, index2))

In [128]:
upper_cat_indexes = set([v for k,v in contained_indexes])
len(contained_indexes), len(upper_cat_indexes)

(678, 456)

In [129]:
biased_indexes = set([v for k,v in duplicated_upper_indexes])
len(biased_indexes)

111

In [130]:
# these should be removed
df_flattened_dd["biased_indexes"] = df_flattened_dd.index.isin(biased_indexes)
dict(df_flattened_dd[df_flattened_dd["biased_indexes"]]['def_page_title'].value_counts())

{'Definition:Adjacent (Graph Theory)': 1,
 'Definition:Parallel (Geometry)': 1,
 'Definition:Rooted Tree': 1,
 'Definition:Right-Hand Derivative': 1,
 'Definition:Regular Representations': 1,
 'Definition:Rank (Linear Algebra)': 1,
 'Definition:Proper Divisor': 1,
 'Definition:Proper Coloring': 1,
 'Definition:Group Product': 1,
 'Definition:Product (Abstract Algebra)': 1,
 'Definition:Power of Element': 1,
 'Definition:Polar Coordinates': 1,
 'Definition:Periodic Function/Real': 1,
 'Definition:Orthogonal (Analytic Geometry)': 1,
 'Definition:Separated by Neighborhoods': 1,
 'Definition:Matrix/Order': 1,
 'Definition:Elastic Modulus': 1,
 'Definition:Ellipse/Minor Axis': 1,
 'Definition:Median (Statistics)': 1,
 'Definition:Topological Manifold': 1,
 'Definition:Ellipse/Major Axis': 1,
 'Definition:Lower Bound of Sequence': 1,
 'Definition:Lower Bound of Mapping': 1,
 'Definition:Lower Bound of Set': 1,
 'Definition:Left-Hand Derivative': 1,
 'Definition:Kernel (Abstract Algebra)': 1,

In [131]:
df_flattened_dd["def_contains_other_def"] = df_flattened_dd.index.isin(upper_cat_indexes)

In [137]:
df_flattened_dd[df_flattened_dd["def_contains_other_def"] & ~df_flattened_dd["biased_indexes"]]

,dd_page_title,term,definition,def_page_title,categories,plain_text_definition,term_in_def,biased_indexes,def_contains_other_def
5,Definition:Absolute,Absolute,=== Definition 1 ===\nLet $x \in \mathbb R$ be...,Definition:Absolute Value,"['Definitions/Absolute Value Function', 'Defin...",=== Definition 1 ===\nLet x ∈ℝ be a real numbe...,True,False,True
12,Definition:Absolute,Absolute,Let $S$ be a sample or a population.\n\nLet $\...,Definition:Frequency (Descriptive Statistics),['Definitions/Frequency (Descriptive Statistic...,Let S be a sample or a population.\n\nLet ω be...,False,False,True
22,Definition:Action,Action,"Let $X$ be a set.\n\nLet $\left( G, \circ \rig...",Definition:Group Action,['Definitions/Group Actions'],"Let X be a set.\n\nLet ( G, ∘) be a group whos...",True,False,True
23,Definition:Action,Action,Let $R$ be a ring.\n\nLet $M$ be an abelian gr...,Definition:Linear Ring Action,"['Definitions/Module Theory', 'Definitions/Lin...",Let R be a ring.\n\nLet M be an abelian group....,True,False,True
35,Definition:Adjacent,Adjacent,"=== Undirected Graph ===\nLet $G = \left( V, E...",Definition:Adjacent (Graph Theory)/Edges,"['Definitions/Edges of Graphs', 'Definitions/A...","=== Undirected Graph ===\nLet G = ( V, E ) be ...",True,False,True
...,...,...,...,...,...,...,...,...,...
1929,Definition:Value,Value,"Let $F$ be a field, such as the field of real ...",Definition:Value of Continued Fraction,['Definitions/Continued Fractions'],"Let F be a field, such as the field of real nu...",True,False,True
1946,Definition:Walk,Walk,A closed walk is a walk whose first vertex is ...,Definition:Walk (Graph Theory)/Closed,['Definitions/Walks'],A closed walk is a walk whose first vertex is ...,True,False,True
1948,Definition:Walk,Walk,"Let $G = \left( V, A \right)$ be a digraph.\n\...",Definition:Directed Walk,"['Definitions/Digraphs', 'Definitions/Walks']","Let G = ( V, A ) be a digraph.\n\n\nA directed...",True,False,True
1973,Definition:Zero,Zero,"Let $\left( S, \circ \right)$ be an algebraic ...",Definition:Zero Element,"['Definitions/Abstract Algebra', 'Definitions/...","Let ( S, ∘) be an algebraic structure.\n\n\n==...",True,False,True


In [168]:
df_no_biased = df_flattened_dd[~df_flattened_dd["biased_indexes"]]

In [169]:
df_no_biased[df_no_biased['term'] == 'Definite']

,dd_page_title,term,definition,def_page_title,categories,plain_text_definition,term_in_def,biased_indexes,def_contains_other_def,term_in_def_page_title
504,Definition:Definite,Definite,Let $\mathbb C$ be the field of complex number...,Definition:Non-Negative Definite Mapping,['Definitions/Hilbert Spaces'],Let ℂ be the field of complex numbers.\n\nLet ...,True,False,False,True
505,Definition:Definite,Definite,"Let $\left[ a \,.\,.\, \right]b$ be a closed...",Definition:Definite Integral,"['Definitions/Definite Integrals', 'Definition...",Let [ a . . ]b be a closed real interval.\n\n...,True,False,True,True


In [170]:
term_cnt = dict(df_no_biased['term'].value_counts())
term_cnt

{'System': 54,
 'Left': 43,
 'Right': 43,
 'Normal': 24,
 'Proper': 23,
 'Harmonic': 22,
 'Closed': 20,
 'Length': 18,
 'Regular': 17,
 'Algebraic': 16,
 'Absolute': 16,
 'Generator': 16,
 'Generated': 16,
 'Mean': 15,
 'Adjacent': 15,
 'Order': 15,
 'Principal': 14,
 'Trivial': 14,
 'Similar': 14,
 'Degree': 14,
 'Closure': 14,
 'Dual': 14,
 'Continuous': 13,
 'Discrete': 13,
 'Compatible': 13,
 'Integral': 13,
 'Homogeneous': 13,
 'Dimension': 12,
 'Conjugate': 12,
 'Null': 12,
 'Limit': 12,
 'Isomorphism': 11,
 'Base': 11,
 'Common': 11,
 'Power': 11,
 'Algebra': 11,
 'Zero': 11,
 'Space': 11,
 'Extension': 10,
 'Section': 10,
 'Complete': 10,
 'Product': 10,
 'Inverse': 10,
 'Free': 9,
 'Directed': 9,
 'Composition': 9,
 'Transformation': 9,
 'Kernel': 9,
 'Degenerate': 9,
 'Altitude': 9,
 'Affine': 9,
 'Index': 9,
 'Bounded': 9,
 'Primitive': 9,
 'Orthogonal': 9,
 'Term': 9,
 'Component': 9,
 'Complex': 8,
 'Irreducible': 8,
 'Connected': 8,
 'Vertex': 8,
 'Simple': 8,
 'Empty': 8

In [217]:
df_no_biased = df_no_biased.drop(df_no_biased[df_no_biased['term']=='Period'].index)

In [219]:
def_set = set(df_no_biased["plain_text_definition"])
pkl_vars(def_set, "data/vars/def_set.pkl")

title_set = set(df_no_biased["def_page_title"])
pkl_vars(title_set, "data/vars/title_set.pkl")

In [220]:
parsed_disamb_list = []
for key, group in df_no_biased.groupby("term"):
    items = group[["plain_text_definition", "def_page_title"]].to_dict(orient="records")
    parsed_disamb_list.append({"term": key, "def_item_list": items})

In [221]:
pkl_vars(parsed_disamb_list,"data/vars/parsed_disamb_list_without===_without_bias.pkl")

In [222]:
parsed_disamb_list = reload_vars("data/vars/parsed_disamb_list_without===_without_bias.pkl")

In [223]:
len(parsed_disamb_list)

343

In [224]:
import random
random.shuffle(parsed_disamb_list)
test_disamb_list = parsed_disamb_list[:68] # int(0.2 * len(parsed_disamb_list)) = 68
train_disamb_list = parsed_disamb_list[68:]
len(train_disamb_list)

275

In [225]:
pkl_vars(train_disamb_list, 'data/vars/train_disamb_list.pkl')
pkl_vars(test_disamb_list, 'data/vars/test_disamb_list.pkl')

In [226]:
test_disamb_list[0]

{'term': 'Boundary',
 'def_item_list': [{'plain_text_definition': ' \n\nFor example, the endpoints of a line segment are its boundaries.\n\n\n=== Containment ===\nA geometric figure is said to be contained by its boundary or boundaries.\n\n ',
   'def_page_title': 'Definition:Boundary (Geometry)'},
  {'plain_text_definition': 'Let T = ( S, τ) be a topological space.\n\nLet H ⊆ S.\n\n\n=== Definition from Closure and Interior ===\nLet T = ( S, τ) be a topological space.\n\nLet H ⊆ S.\n\n\nThe boundary of H consists of all the points in the closure of H which are not in the interior of H.\n\nThus, the boundary of H is defined as:\n:∂ H := H^- ∖ H^∘\nwhere H^- denotes the closure and H^∘ the interior of H.\n\n=== Definition from Neighborhood ===\nLet T = ( S, τ) be a topological space.\n\nLet H ⊆ S.\n\n\n \n\nx ∈ S is a boundary point of H  if and only if  every neighborhood N of x satisfies:\n:H ∩ N ∅\nand\n:H∩ N ∅\nwhere H is the complement of H in S.\n\nThe boundary of H consists of al

### make  dict

In [227]:
# the kb is a dictionary of notion and definition page titles, each title contains the term and its domain
def make_def_kb(def_disamb_list):
    def_kb = dict()
    for x in def_disamb_list:
        term = x['term']
        titles = [def_item['def_page_title'] for def_item in x['def_item_list']]
        def_kb[term] = titles
    return def_kb

def_disamb_kb = make_def_kb(parsed_disamb_list)
len(def_disamb_kb)

343

In [228]:
pkl_vars(def_disamb_kb, 'data/vars/def_disamb_kb.pkl')

In [229]:
def flatten_def_disamb_list(disamb_list):
    flattened_dd_list = []
    for li in disamb_list:
        for def_item in li['def_item_list']:
            data = {
                'title': def_item['def_page_title'],
                'term': li['term'],
                'definition': def_item['plain_text_definition'],
                #'def_page_title': def_item['title'],
                #'categories': def_item['categories']
            }
            flattened_dd_list.append(data)
    return flattened_dd_list

In [230]:
flattened_test_disam_list = flatten_def_disamb_list(test_disamb_list)
len(flattened_test_disam_list)

433

In [231]:
flattened_train_disamb_list = flatten_def_disamb_list(train_disamb_list)
len(flattened_train_disamb_list)

1436

In [232]:
pkl_vars(flattened_train_disamb_list, 'data/vars/flattened_train_disamb_list.pkl')

In [233]:
pkl_vars(flattened_test_disam_list, 'data/vars/flattened_test_disam_list.pkl')

In [234]:
flattened_test_disam_list = reload_vars('data/vars/flattened_test_disam_list.pkl')

In [235]:
pkl_vars(flattened_test_disam_list[:9], "data/vars/9_test_sample_list.pkl")

In [236]:
df_flattened_test_disam_list = pd.DataFrame(flattened_test_disam_list)
df_flattened_test_disam_list.to_csv("data/SP_CLS/df_flattened_test_disam_list.csv", index=False)

In [237]:
flattened_train_disamb_list = reload_vars('data/vars/flattened_train_disamb_list.pkl')
df_flattened_train_disam_list = pd.DataFrame(flattened_train_disamb_list)
df_flattened_train_disam_list.to_csv("data/SP_CLS/df_flattened_train_disam_list.csv", index=False)

In [238]:
df_flattened_test_disam_list[:9].to_csv("data/SP_CLS/9_test_samples.csv", index=False)
df_flattened_train_disam_list[:5].to_csv("data/SP_CLS/5_samples.csv", index=False)

### GLADIS data samples

# defining DefDisambiguationBERT

## model for next sentence prediction 

In [4]:
#Copied from https://github.com/tigerchen52/GLADIS/blob/master/source/acrobert.py
def_disamb_kb = reload_vars('data/vars/def_disamb_kb.pkl')

class DefDisambiguationBERT(nn.Module):
    #margin: the margin value lambda in the triplet loss'
    def __init__(self, model_name="bert-base-uncased", from_tf=False, device='cpu', margin=0.2):
        super().__init__()
        self.model_name=model_name
        self.device = device
        self.model = BertForNextSentencePrediction.from_pretrained(model_name, from_tf=from_tf)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.margin = margin

    def forward(self, pos_x, masked_pos_x=None, neg_x=None, train=True):
        loss, scores = 0.0, 0.0
        if train:
            pos_samples = self.tokenizer(pos_x, padding=True, return_tensors='pt', truncation=True)["input_ids"]
            neg_x = self.tokenizer(neg_x, padding=True, return_tensors='pt', truncation=True)["input_ids"]

            pos_samples = pos_samples.to(self.device)
            neg_x = neg_x.to(self.device)

            pos_outputs = self.model(pos_samples).logits
            neg_outputs = self.model(neg_x).logits
            pos_scores = 1 - nn.Softmax(dim=1)(pos_outputs)[:, 0]
            neg_scores = 1 - nn.Softmax(dim=1)(neg_outputs)[:, 0]
            loss = triplet_loss(pos_scores, neg_scores, self.margin)

        else:
            samples = self.tokenizer(pos_x, padding=True, return_tensors='pt', truncation=True)["input_ids"]
            samples = samples.to(self.device)
            outputs = self.model(samples).logits
            scores = nn.Softmax(dim=1)(outputs)[:, 0]


        return loss if train else scores
    
        
def triplet_loss(pos_score, neg_score, margin=0.2):
    losses = torch.relu(pos_score - neg_score + margin)
    return losses.mean()


def softmax(elements):
    total = sum([exp(e) for e in elements])
    return exp(elements[0]) / total

# todo: cache pairwised nsp scores...
def cal_nsp_score(model, tokenizer, titles, definition, batch_size):
    ps = list()
    for index in range(0, len(titles), batch_size):
        batch_tt = titles[index:index + batch_size]
        batch_df = [definition] * len(batch_tt)
        encoding = tokenizer(batch_tt, batch_df, return_tensors="pt", padding=True, truncation=True, max_length=400)#.to(device)
        outputs = model(**encoding)
        logits = outputs.logits#.cpu().detach().numpy()
        p = [softmax(lg) for lg in logits]
        ps.extend(p)
    return ps

## model for embedding similarity

In [38]:
class SentenceEncoderModel(nn.Module):
    def __init__(self, model_name="bert-base-uncased", #check_point_path=None,
                 from_tf=False, device='cpu'):
        super().__init__()
        self.model_name=model_name
        self.device = device
        self.model = AutoModel.from_pretrained(model_name, from_tf=from_tf)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # cache embeddings to  avoid repeated calculations
        def_emb_dict_path = f"data/vars/{model_name.split('/')[-1]}_def_emb_dict.pkl"            
        if not os.path.isfile(def_emb_dict_path):
            defs = list(reload_vars("data/vars/def_set.pkl"))
            def_embs = get_embeddings(self.model, self.tokenizer, defs)
            def_emb_dict = dict()
            for index, definition in enumerate(defs):
                def_emb_dict[definition] = def_embs[index]
            pkl_vars(def_emb_dict, def_emb_dict_path)
        else:
            def_emb_dict = reload_vars(def_emb_dict_path)           
        self.def_emb_dict = def_emb_dict

        title_emb_dict_path = f"data/vars/{model_name.split('/')[-1]}_title_emb_dict_path.pkl"
        if not os.path.isfile(title_emb_dict_path):
            titles = list(reload_vars("data/vars/title_set.pkl"))
            tt_embs = get_embeddings(self.model, self.tokenizer, titles)
            title_emb_dict = dict()
            for index, tt in enumerate(titles):
                title_emb_dict[tt] = tt_embs[index]
            pkl_vars(title_emb_dict, title_emb_dict_path)
        else:
            title_emb_dict = reload_vars(title_emb_dict_path)
        self.title_emb_dict = title_emb_dict
        
        
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def get_embeddings(model, tokenizer, sentences):
    batch_tokenizer_output = tokenizer(
        sentences,
        max_length=400,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        batch_model_output = model(**batch_tokenizer_output, output_hidden_states=True)
    token_embeddings = batch_model_output.hidden_states[-1]
    return mean_pooling(batch_model_output, batch_tokenizer_output.attention_mask).tolist() # convert .tolist() to save storage spaces
        

def cal_sim_score(sem_model, titles, definition):
    if definition in sem_model.def_emb_dict:
        def_emb = sem_model.def_emb_dict[definition]
    else:
        def_emb = get_embeddings(sem_model.model, sem_model.tokenizer, [definition])[0]

    tt_embs = []
    for tt in titles: 
        if tt in sem_model.title_emb_dict:
            tt_emb = sem_model.title_emb_dict[tt]
        else:
            tt_emb = get_embeddings(sem_model.model, sem_model.tokenizer, [tt])[0]
        tt_embs.append(tt_emb)
    
    ps = [F.cosine_similarity(torch.Tensor(def_emb), torch.Tensor(tt_emb), dim = -1) for tt_emb in tt_embs]
    return ps

## utils

In [6]:
# adpated from https://github.com/tigerchen52/GLADIS/blob/master/source/utils.py 

#def_disamb_kb = reload_vars('data/vars/def_disamb_kb.pkl')
def get_candidate(def_disamb_kb, term):
    return def_disamb_kb[term]

class TextData(Dataset):
    def __init__(self, data):
        self.all_term = data['term']
        self.all_title = data['title']
        self.all_definition = data['definition']

    def __len__(self):
        return len(self.all_term)

    def __getitem__(self, idx):
        return self.all_term[idx], self.all_title[idx], self.all_definition[idx]

def load_pretrain(train_data_path='data/SP_CLS/df_flattened_train_disam_list.csv'):
    flattened_train_disamb_list = pd.read_csv(train_data_path)
    all_term = list(flattened_train_disamb_list['term'])
    all_title =  list(flattened_train_disamb_list['title'])
    all_definition = list(flattened_train_disamb_list['definition'])
    return {'term': all_term, 'title': all_title, 'definition': all_definition}

class DefDisambiguationBertLoader():
    def __init__(self, batch_size, tokenizer, kb, shuffle=True, masked_prob=0.15, hard_num=2):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.tokenizer = tokenizer
        self.masked_prob = masked_prob
        self.hard_num = hard_num
        self.kb = kb
        self.all_titles = list()
        for vs in self.kb.values():
            self.all_titles.extend(list(vs))

    def select_negative(self, target):
        selected, flag, max_time = None, True, 10
        if target in self.kb:
            title_candidates = self.kb[target]
            if len(title_candidates) == 1:
                title_candidates = self.all_titles
        else:
            title_candidates = self.all_titles
        attempt = 0
        while flag and attempt < max_time:
            attempt += 1
            selected = random.choice(title_candidates)
            if selected != target:
                flag = False
        if attempt == max_time:
            selected = random.choice(self.all_titles)
        return selected

    def collate_fn(self, batch_data):
        batch_term, batch_title, batch_definition = list(zip(*batch_data))
        pos_samples, neg_samples, masked_pos_samples = list(),  list(), list()
        for _ in range(self.hard_num):
            temp_pos_samples = [batch_title[index] + ' [SEP] ' + batch_definition[index] for index in range(len(batch_title))]
            neg_titles = [self.select_negative(t) for t in batch_term]
            temp_neg_samples = [neg_titles[index] + ' [SEP] ' + batch_definition[index] for index in range(len(batch_title))]
            temp_masked_pos_samples = [batch_title[index] + ' [SEP] ' + batch_definition[index] for index in range(len(batch_title))]

            pos_samples.extend(temp_pos_samples)
            neg_samples.extend(temp_neg_samples)
            masked_pos_samples.extend(temp_masked_pos_samples)
        return pos_samples,  masked_pos_samples,  neg_samples

    def __call__(self, data_path):
        dataset = load_pretrain(train_data_path=data_path)
        logger.info('loaded dataset, sample = {a}'.format(a=len(dataset['term'])))
        dataset = TextData(dataset)
        train_iterator = DataLoader(dataset=dataset, batch_size=self.batch_size // (2 * self.hard_num), shuffle=self.shuffle,
                                    collate_fn=self.collate_fn)
        return train_iterator

## Sentence Pair CLS Training

In [243]:
def train_scp(device = device, batch_size = 16, kb = def_disamb_kb, from_tf= False, shuffle = True,
         hard_neg_numbers = 2, train_data_path = 'data/SP_CLS/df_flattened_train_disam_list.csv',
         lr=2e-5, lr_decay = 0.95, epoch_num = 30, loss_check_step =1, check_step = 1000, 
         output_model_path="trained_models/" ):
    
    model = DefDisambiguationBERT(device=device,from_tf=from_tf)
    model.to(device)
    loader = DefDisambiguationBertLoader(batch_size, tokenizer=model.tokenizer, kb=kb, shuffle=shuffle, hard_num=hard_neg_numbers)

    train_loader = loader(train_data_path)
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logger.info(trainable_num)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

    max_f1, max_epoch = 0.0, 0
    for e in range(epoch_num):
        epoch_loss = 0
        batch_num = 0

        for pos_samples, masked_pos_samples, neg_samples in train_loader:
            model.train()
            optimizer.zero_grad()

            if batch_num % loss_check_step == 0 and batch_num != 0:
                logger.info('sample = {b}, loss = {a}'.format(a=epoch_loss / batch_num, b=batch_num * batch_size))

            if batch_num % check_step == 0 and batch_num != 0:
                for g in optimizer.param_groups:
                    g['lr'] *= lr_decay
            #if batch_num % check_step == 0 and batch_num != 0:
            #    temp_path = output_model_path.format(a=str(e + 1) + '_' + str(batch_num))
            #    torch.save(model.state_dict(), temp_path)
            loss = model(pos_samples, masked_pos_samples, neg_samples)

            # backward
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            batch_num += 1

        scheduler.step()
        temp_path = f"{output_model_path}_{model.model_name.split('/')[-1]}_{e+1}_epoch.pt"
        logger.info('the pre-training finished, saving model, path = {a}'.format(a=temp_path))
        torch.save(model.state_dict(), temp_path)
    return max_f1, max_epoch


In [244]:
%%time
train_scp(train_data_path="data/SP_CLS/5_samples.csv", epoch_num = 3, batch_size=4)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 4.15 s, sys: 1.23 s, total: 5.38 s
Wall time: 3.67 s


(0.0, 0)

In [ ]:
%%time
train_scp(train_data_path = 'data/SP_CLS/df_flattened_train_disam_list.csv', epoch_num = 50, batch_size=16, lr=1e-5)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Evaluation

In [7]:
# adpated from https://github.com/tigerchen52/GLADIS/blob/master/source/evaluation.py
from sklearn.metrics import f1_score
from collections import OrderedDict


def transform_to_index(trues, preds):

    true_map = dict([(title, index) for index, title in enumerate(list(OrderedDict.fromkeys(trues)))])
    tag_cnt = len(true_map)
    true_index = [true_map[t] for t in trues]

    for pred in preds:
        if pred not in true_map:
            true_map[pred] = len(true_map)
    pred_index = [true_map[pred] for pred in preds]
    return true_index, pred_index, tag_cnt


def macro_f1(trues, preds):
    true_index, pred_index, tag_cnt = transform_to_index(trues, preds)
    f1_scores = f1_score(true_index, pred_index, average=None)[:tag_cnt]
    macro_f1_score = sum(f1_scores) / len(f1_scores)
    return macro_f1_score


def accuracy(trues, preds):
    acc_cnt = 0
    for index, true in enumerate(trues):
        pred = preds[index]
        if pred == true:
            acc_cnt += 1
    acc = acc_cnt * 1.0 / len(trues)
    return acc

def predict(task_model, term, definition, batch_size, dd_kb=def_disamb_kb, cal="nsp"):
    titles = get_candidate(dd_kb, term)
    #titles = [str.lower(can) for can in titles]
    if "nsp" == cal:
        scores = cal_nsp_score(task_model.model, task_model.tokenizer, titles, definition, batch_size)
        max_index = np.argmax(scores)
    else:
        scores = cal_sim_score(task_model, titles, definition)
        max_index = np.argmax(scores)
    return titles[max_index]

# evaluate prediction based on nsp and kb
def eval_pred(task_model, flattened_dd_list_path, batch_size, dd_kb=def_disamb_kb, cal="nsp",train=True):
    data = reload_vars(flattened_dd_list_path)
    true_labels, pred_labels = list(), list()
    for index, sample in enumerate(data):
        if index % 100 == 0:
            logger.info('processing {a} lines '.format(a=index))
        if train and index > 200: break
        term = sample['term']
        title = sample['title']
        definition = sample['definition']
        pred = predict(task_model, term, definition, batch_size, dd_kb=def_disamb_kb, cal=cal)
        true_labels.append(title)
        pred_labels.append(pred)
    df_res = pd.DataFrame(data)
    df_res['pred'] = pred_labels
    df_res.to_csv(f"data/res/{task_model.model_name.split('/')[-1]}_{cal}_{'train' if 'train' in flattened_dd_list_path else 'test'}.csv")
    macro_f1_score = macro_f1(true_labels, pred_labels)
    acc = accuracy(true_labels, pred_labels)
    return macro_f1_score, acc


# Evaluate NSP with NSP-finetuned LMs

In [48]:
def run_eval(device=device, batch_size=4, min_epoch_num = 0, epoch_num=3, 
             test_data_path="data/vars/flattened_test_disam_list.pkl",
             output_model_path="trained_models/" ):
    for epoch in range(min_epoch_num, epoch_num):
        model = DefDisambiguationBERT(device=device)
        model_path = f"{output_model_path}_{model.model_name.split('/')[-1]}_{epoch+1}_epoch.pt"
        model.model_name = f"{model.model_name}_{epoch+1}_ep"
        model.load_state_dict(torch.load(model_path, map_location=device))
        macro_f1_score, acc = eval_pred(model, test_data_path,
                                batch_size = batch_size, dd_kb=def_disamb_kb, train=False) 
        logger.info('Epoch {e}: F1: {a}, ACC: {b}'.format(e=epoch+1, a=macro_f1_score, b=acc))

In [46]:
for epoch in range(1, 3):print(epoch)

1
2


In [260]:
%%time
run_eval(device=device, batch_size=32, epoch_num=1, test_data_path="data/vars/9_test_sample_list.pkl")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 10min 49s, sys: 8.39 s, total: 10min 57s
Wall time: 14.8 s


In [ ]:
%%time
run_eval(device=device, batch_size=4, epoch_num=50)

In [ ]:
%%time
run_eval(device=device, batch_size=16, min_epoch_num=10, epoch_num=50, test_data_path="data/vars/flattened_train_disamb_list.pkl")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Evaluate NSP with OOB LMs
Note: To cache the pairwised NSP calculations

In [115]:
%%time
cc_model = DefDisambiguationBERT("InriaValda/cc_math_bert_ep10", from_tf=True, device= device)
macro_f1_score, acc = eval_pred(cc_model, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 32, dd_kb=def_disamb_kb, train=False) 
macro_f1_score, acc 

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 2h 58min 16s, sys: 8min 32s, total: 3h 6min 49s
Wall time: 4min 7s


(0.08303624480095069, 0.13031914893617022)

In [ ]:
%%time
cc_model = DefDisambiguationBERT("InriaValda/cc_math_bert_ep10", from_tf=True, device= device)
macro_f1_score, acc = eval_pred(cc_model, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False) 
macro_f1_score, acc 

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-10-22 15:15:38.327174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-22 15:15:38.486482: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 15:15:39.140395: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cann

In [8]:
%%time 
BERT_OOB = DefDisambiguationBERT(device= device)
macro_f1_score, acc = eval_pred(BERT_OOB, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 32, dd_kb=def_disamb_kb, train=False) 
macro_f1_score, acc

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 5h 3min 11s, sys: 23min 38s, total: 5h 26min 50s
Wall time: 7min 23s


(0.8089622641509436, 0.8475750577367206)

In [11]:
%%time
macro_f1_score, acc = eval_pred(BERT_OOB, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 32, dd_kb=def_disamb_kb, train=False) 
macro_f1_score, acc

CPU times: user 11h 22min 18s, sys: 38min 59s, total: 12h 1min 18s
Wall time: 15min 46s


(0.7982990712720429, 0.8391364902506964)

# Evaluate embedding similarity with NSP-finetuned LMs


In [42]:
def run_eval_sim(device=device, batch_size=4, check_point_nums=[0], test_data_path="data/vars/flattened_test_disam_list.pkl"):
    test_data_str = test_data_path.split('/')[-1].split('.pkl')[0]
    for epoch in check_point_nums:
        
        ft_model_name = f"ft_models/bert-base-uncased_nsp_{epoch+1}_ep"
        # check if we have alread save the bert model
        if not os.path.isdir(ft_model_name):
            dd_model = DefDisambiguationBERT(device=device)
            dd_model_path = f"{output_model_path}_{dd_model.model_name.split('/')[-1]}_{epoch+1}_epoch.pt"
            dd_model.load_state_dict(torch.load(dd_model_path, map_location=device))
            dd_model.model.save_pretrained(ft_model_name)
            dd_model.tokenizer.save_pretrained(ft_model_name)
            del dd_model
            
        sem_model = SentenceEncoderModel(ft_model_name, device= device)

        macro_f1_score, acc = eval_pred(sem_model, test_data_path, dd_kb=def_disamb_kb, train=False,
                                       cal="sim", batch_size=4)  
        logger.info('Evaluate emb sim of Epoch {e} on {test_data_str} set: F1: {a}, ACC: {b}'.format(e=epoch+1, test_data_str = test_data_str, a=macro_f1_score, b=acc))

In [45]:
%%time
run_eval_sim(device=device, batch_size=4, check_point_nums=[23,42], test_data_path="data/vars/flattened_test_disam_list.pkl")
run_eval_sim(device=device, batch_size=4, check_point_nums=[23,42], test_data_path="data/vars/flattened_train_disamb_list.pkl")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 7h 31min 44s, sys: 3h 34min 35s, total: 11h 6min 20s
Wall time: 19min 21s


# Evaluate embedding similarity with OOB LMs
Notes: without the pickled sentence-embedding dictionaries, each 768-dimension sentence bert model takes ~ 11 minutes to calculate all the embeddings once.

In [39]:
%%time
cc_sem_model = SentenceEncoderModel("InriaValda/cc_math_bert_ep10", from_tf=True, device= device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 1.37 s, sys: 608 ms, total: 1.97 s
Wall time: 1.54 s


In [253]:
%%time
macro_f1_score, acc = eval_pred(cc_sem_model, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

CPU times: user 1.4 s, sys: 36.3 ms, total: 1.44 s
Wall time: 1.44 s


(0.2820924882245639, 0.3787528868360277)

In [254]:
%%time
macro_f1_score, acc = eval_pred(cc_sem_model, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 32, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

CPU times: user 2.43 s, sys: 0 ns, total: 2.43 s
Wall time: 2.43 s


(0.3560199829675193, 0.45403899721448465)

In [248]:
%%time
mini_sem_model = SentenceEncoderModel("sentence-transformers/all-MiniLM-L6-v2", device= device)

macro_f1_score, acc = eval_pred(mini_sem_model, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 1.97 s, sys: 176 ms, total: 2.15 s
Wall time: 1.3 s


(0.9008647798742141, 0.9237875288683602)

In [249]:
macro_f1_score, acc = eval_pred(mini_sem_model, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

(0.8813732651570487, 0.9101671309192201)

In [250]:
%%time
bert_sem_model = SentenceEncoderModel("bert-base-uncased", device= device)

macro_f1_score, acc = eval_pred(bert_sem_model, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 3.81 s, sys: 592 ms, total: 4.4 s
Wall time: 1.95 s


(0.2611149736068411, 0.3533487297921478)

In [252]:
%%time
macro_f1_score, acc = eval_pred(bert_sem_model, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

CPU times: user 2.37 s, sys: 14.9 ms, total: 2.39 s
Wall time: 2.39 s


(0.3019173019173022, 0.4025069637883008)

In [255]:
%%time
MLM_arXiv_sem_model = SentenceEncoderModel("math-similarity/Bert-MLM_arXiv", device= device)

macro_f1_score, acc = eval_pred(MLM_arXiv_sem_model, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 4.15 s, sys: 767 ms, total: 4.92 s
Wall time: 2.08 s


(0.28279607465114975, 0.3833718244803695)

In [257]:
macro_f1_score, acc = eval_pred(MLM_arXiv_sem_model, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

(0.32845973438183346, 0.4268802228412256)

In [258]:
%%time
zbmath_sem_model = SentenceEncoderModel("math-similarity/Bert-MLM_arXiv-MP-class_zbMath", device= device)

macro_f1_score, acc = eval_pred(zbmath_sem_model, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 3.59 s, sys: 944 ms, total: 4.53 s
Wall time: 2.08 s


(0.43801662174303685, 0.5219399538106235)

In [259]:
macro_f1_score, acc = eval_pred(zbmath_sem_model, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

(0.5399012561174714, 0.6149025069637883)

In [17]:
%%time
mini_sem_model_12 = SentenceEncoderModel("sentence-transformers/all-MiniLM-L12-v2", device= device)
 
macro_f1_score, acc = eval_pred(mini_sem_model_12, "data/vars/flattened_test_disam_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 2.51 s, sys: 270 ms, total: 2.78 s
Wall time: 1.36 s


(0.9118710691823901, 0.9330254041570438)

In [19]:
macro_f1_score, acc = eval_pred(mini_sem_model_12, "data/vars/flattened_train_disamb_list.pkl",
                                batch_size = 4, dd_kb=def_disamb_kb, train=False,
                               cal="sim") 
macro_f1_score, acc

(0.8944728512296076, 0.9206128133704735)

In [166]:
len(cc_sem_model.title_emb_dict['Definition:Field Zero'])

768

In [188]:
cal_sim_score(cc_sem_model, [title0, 'blabla'], def0)

[tensor(0.6233), tensor(0.5268)]

In [178]:
title02 = 'Definition:Field Zero2'
def02 = 'A2 dynamical system is a non-linear system in which a function describes the time dependence of a point in a geometrical space.\n\n\n=== Flow ===\nIn a dynamical system, a set of time-dependent equations is known as flow.\n\n '

In [189]:
embs = get_embeddings(cc_sem_model.model, cc_sem_model.tokenizer, [title0, 'blabla'])